## Required Imports

In [1]:
from z3 import *
from datetime import datetime
import numpy as np
import pickle

## Helper functions

In [2]:
def get_mini_grid(array, i, j, size):
    return [ L[i * size + y][j * size + x] for y in range(size) for x in range(size) ]


def get_grid(puzzler):
    size = len(puzzler)
    m_size = int(size**(1/2))
    char_len = len(str(size))
    horizontal = '+' + ''.join([f'{"-"*(m_size * (char_len + 1) + 1)}+' for _ in range(m_size)])
    output = ''
    for r in range(size):
        if r % m_size == 0:
            output = output + f'{horizontal}\n'
        line = [' '.join([f'{_:>{char_len}}' for _ in puzzler[r][(c * m_size):(c * m_size+ m_size)]]) 
                for c in range(0, m_size)]
        line_with_divider = ' | '.join([_ for _ in line])
        output = output + f'| {line_with_divider} |\n'
    return f'{output}{horizontal}'.replace(f'{0:>{char_len}}',f'{"∙":>{char_len}}')


def exec_time(start, end):
    return str(end - start)


def get_solution(model, L):
    return [ [ model.eval(L[i][j]).as_long() for j in range(size) ] for i in range(size) ]


def side_by_side(grid1, grid2, label_1=None, label_2=None, spaces=5):
    s1, s2 = get_grid(puzzle), get_grid(solution)
    separator = ' ' * spaces
    output = [f'{l}{separator}{r}' for (l, r) in zip(s1.split('\n'), s2.split('\n'))]
    if label_1 is not None and label_2 is not None:
        s1_len = len(s1.split('\n')[0])
        out = f'{label_1:<{s1_len}}{separator}{label_2}'
        output.insert(0, out)
    return '\n'.join(output)

## Configuration
1. Set puzzle
2. Set size and mini box size ($\textit{m_size}$)

In [3]:
# Set the puzzle
puzzle_16x16 = (
    (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1),
    (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (0,0,0,0,0,0,0,3,0,0,0,13,0,0,0,4),
    (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (0,0,0,0,0,0,15,0,0,0,0,0,0,0,0,0),
    (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (0,0,0,0,0,0,0,0,0,0,0,0,0,7,0,0),
    (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0),
    (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (0,0,0,0,0,0,0,0,0,0,9,0,0,0,0,8),
    (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
    (0,0,0,7,0,0,0,0,0,0,0,0,12,0,0,0),
)

puzzle_25x25 = [
    (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
    (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
    (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0),
    (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
    (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0),
    (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
    (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
    (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
    (0, 0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 24),
    (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
    (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 11, 0, 0),
    (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
    (0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
    (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
    (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
    (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
    (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 0, 0, 0, 0, 0, 0, 0),
    (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
    (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
    (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
    (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
    (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
    (0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0),
    (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
    (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
]

puzzle = puzzle_16x16

# Set the size and m_size
size = len(puzzle)
m_size = int(size**(0.5))

## Constraints

As a recap, here are the restraints that my Sudoku Z3 solver requires

* <span style="color:green">Every mini box (grid size 3x3) can only contain the numbers 1 to 9.</span> (<span style="color:red">Constraint 1</span>)
* Every mini box cannot have duplicate numbers.  
* <span style="color:green">Every vertical column can only contain the numbers 1 to 9.</span> (<span style="color:red">Constraint 2</span>)
* Every vertical column cannot have duplicate numbers.  
* <span style="color:green">Every horizontal row can only contain the numbers 1 to 9.</span> (<span style="color:red">Constraint 3</span>)
* Every horizontal row cannot have duplicate numbers.  
* <span style="color:green">Every mini box can only contain the numbers 1 to 9.</span> (<span style="color:red">Constraint 4</span>)  
* Every box cannot have duplicate numbers.  

## Solution

### Setup
Before we start working on the solution, we must create references part references that Z3 will understand:

In [4]:
L = []
for row in range(size):
    L.append( [Int(f'L_{row}{col}') for col in range(size)] )

### Constraints

Now we can work on creating the solution for constraints 1-4 mentioned above:

In [5]:
# Constraints every cells to the number 1 - {size}, where size is the size of the Sudoku as in sizeXsize
constraint_1 = [ And( L[row][col] >= 1, L[row][col] <= size)  for col in range(size) for row in range(size) ]

# Constraint on columns, so that each column has to be distinct
constraint_2 = []
for col in range(size):
    constraint_2.append( Distinct( [L[row][col] for row in range(size)] ) )

# Constraint on rows, so that each row has to be distinct
constraint_3 = []
for row in range(size):
    constraint_3.append( Distinct( [L[row][col] for col in range(size)] ) )
    
constraint_4 = [ Distinct( get_mini_grid(L, y, x, m_size) )  for y in range(m_size) for x in range(m_size) ]  

### Add known values

We now add the known values from the puzzle into the references in $\textbf{L}$.  Since the zeros in the puzzle are outside constraint 1, we need to only add the non-zero references

In [6]:
known_values = [L[i][j] == puzzle[i][j] for i in range(size) for j in range(size) if puzzle[i][j]]

### Z3 Solver

We now create the Z3 solver, add the constraints, and the known values, and ask it to solve it for us, and explicited mention is the model was SAT or not.

In [7]:
# Create Z3 solver
s = Solver()

# Add the constraints
s.add(constraint_1)
s.add(constraint_2)
s.add(constraint_3)
s.add(constraint_4)

# Add the known values
s.add(known_values)

# Ask the solver for a solution and record execution time
start = datetime.now()
status = str(s.check())
end = datetime.now()

if status == 'unsat':
    print('Model is UNSAT')
else:
    solution = get_solution(s.model(), L)
    run_time = 'Running time: ' + exec_time(start, end)
    print(side_by_side(puzzle, solution, 'Original puzzle:', f'{run_time}:'))

Original puzzle:                                              Running time: 0:00:25.828756:
+-------------+-------------+-------------+-------------+     +-------------+-------------+-------------+-------------+
|  ∙  ∙  ∙  ∙ |  ∙  ∙  ∙  ∙ |  ∙  ∙  ∙  ∙ |  ∙  ∙  ∙  1 |     |  3 14  5  2 | 16  4 13 12 | 15  9  8  7 | 11  6 10  1 |
|  ∙  ∙  ∙  ∙ |  ∙  ∙  ∙  ∙ |  ∙  ∙  ∙  ∙ |  ∙  ∙  ∙  ∙ |     |  6  9 13 16 | 14  8  1 15 | 12  4 10 11 |  3  5  2  7 |
|  ∙  ∙  ∙  ∙ |  ∙  ∙  ∙  3 |  ∙  ∙  ∙ 13 |  ∙  ∙  ∙  4 |     | 11 15  7  8 |  5  9 10  3 |  2  6  1 13 | 14 12 16  4 |
|  ∙  ∙  ∙  ∙ |  ∙  ∙  ∙  ∙ |  ∙  ∙  ∙  ∙ |  ∙  ∙  ∙  ∙ |     |  1 10  4 12 |  7  6  2 11 |  3 14  5 16 |  8 15  9 13 |
+-------------+-------------+-------------+-------------+     +-------------+-------------+-------------+-------------+
|  ∙  ∙  1  ∙ |  ∙  ∙  ∙  ∙ |  ∙  ∙  ∙  ∙ |  ∙  ∙  ∙  ∙ |     | 15  6  1  3 |  8  2  4 14 |  5 11  7 10 |  9 16 13 12 |
|  ∙  ∙  ∙  ∙ |  ∙  ∙  ∙  ∙ |  ∙  ∙  ∙  ∙ |  ∙  ∙  ∙  ∙ |     |  5  

## Verification

One of the biggest problems of a Sudoku puzzle is that a Sudoku puzzle might contain more than one solution.  Given a Sudoku puzzle with lots of known values, then the possibilities of the puzzle changing are minimizing.  The opposite is true: given a Sudoku puzzle with very few known values, the possibilities of having various solution increases.  This can be exemplified by an empty puzzle, the possiblities appear to seem endless because a different starting point will yield a different puzzle. So how do we go about verifying that the solution is correct?

We do it manually.  The way we do it manually is by using the constraint rules, but verified independently.  In this case, I am using NumPy. There are two main functions that I create to verify the constraints:

| Function | Description |
|:--------:|-------------|
|`sudoku_distinct` | Checks that every row or column is within the range ['1', '`size`'], verifies that the numbers are consecutive returning either True or False, and ultimately returns a conjunction of all the rows/columns. The paramter `which` is used to specify row/column by passing `which='row'` or `which='col'`, respectively. |
|`sudoku_minigrids`| Creates mini grids from the full solution.  For each mini grid, it calls the function `sudoku_distinct` and ultimately returns a conjunction of all the mini grids. |

Those functions follow:

### Verification helper functions

In [8]:
def sudoku_distinct(puzzle, which='row'):
    if which not in ['col', 'row']:
        raise Exception(f'Invalid parameter: which')
    grid = puzzle if isinstance(puzzle, np.ndarray) else np.array(puzzle)
    size = grid.shape[1]
    grid = grid if which == 'row' else grid.T
    distinct_row = np.arange(grid[0].min(), grid[0].max() + 1)
    condition_1 = np.logical_and(grid >= 1, grid <= size).all()
    condition_3 = np.all(np.sort(grid, axis=1, kind=None, order=None) == distinct_row)
    return np.all([condition_1, condition_3])


def sudoku_minigrids(puzzle):
    grid = puzzle if isinstance(puzzle, np.ndarray) else np.array(puzzle)
    size = grid.shape[0]
    m_size = int(size**(0.5))
    minigrids = []
    for i in range(m_size):
        for j in range(m_size):
            minigrids.append(grid[(i*m_size):(i*m_size + m_size), (j*m_size):(j*m_size + m_size)].reshape((-1,size)))
    return np.all([sudoku_distinct(_) for _ in minigrids])

In [11]:
if 'solution' not in locals():
    print('Model was UNSAT, therefore, there\'s no need for verification')
else:     
    rows = sudoku_distinct(solution, which = 'row')
    cols = sudoku_distinct(solution, which = 'col')
    mini_grids = sudoku_minigrids(solution)
    verification = rows & cols & mini_grids
    print(f'This sudoku solution follows the constraints: {verification}')

This sudoku solution follows the constraints: True
